In [1]:
import pandas as pd
import math
import numpy as np
import bokeh
import xyzservices.providers as xyz
import json
from bokeh import events
from bokeh.io import show, save, output_file, output_notebook
from bokeh.layouts import row, column
from bokeh.plotting import figure, show, save
from bokeh.tile_providers import get_provider, Vendors
from bokeh.models import GeoJSONDataSource, ColumnDataSource, HoverTool, Div
from bokeh.models.callbacks import CustomJS
import pyproj
from pyproj import Transformer
output_notebook()

Loading BokehJS ...

In [2]:
df = pd.read_csv("tstationinfo.csv")

In [3]:
def latlon_to_webmercator(lat, lon):
    R = 6378137  # Earth radius in meters
    x = R * math.radians(lon)
    y = R * math.log(math.tan(math.pi / 4 + math.radians(lat) / 2))
    return x, y

df[['long_merc', 'lat_merc']] = df.apply(lambda row: latlon_to_webmercator(row['latitude'], row['longitude']), axis=1, result_type='expand')

In [4]:
df.head()

,id,name,muni_name,muni_id,latitude,longitude,community_type_id,community_type_description,subcommunity_type_id,subcommunity_type_description,...,etod_sub3d,etod_total,etod_type,line_descr,ov_emppipe,ex_emppipe,ov_hupipe,ex_hupipe,long_merc,lat_merc
0,1,21 DRY DOCK AVE,BOSTON,35,42.344721,-71.031526,1,Inner Core,1,Metropolitan Core Communities,...,7,30,Transit-Related,Silver Line SL2,1435.678810,0.0000,0.0,0.0,-7.907193e+06,5.212758e+06
1,2,25 DRY DOCK AVE,BOSTON,35,42.344749,-71.029960,1,Inner Core,1,Metropolitan Core Communities,...,7,30,Transit-Related,Silver Line SL2,1330.224265,569.8125,0.0,0.0,-7.907019e+06,5.212762e+06
2,3,570 BROADWAY @ ELMWOOD AVE,REVERE,248,42.411900,-71.011100,1,Inner Core,1,Metropolitan Core Communities,...,8,27,Transit-Related,116,0.000000,0.0000,0.0,0.0,-7.904919e+06,5.222881e+06
3,4,633 PARK AVE,REVERE,248,42.414400,-71.028600,1,Inner Core,1,Metropolitan Core Communities,...,7,24,Transit-Related,111,0.000000,0.0000,0.0,0.0,-7.906868e+06,5.223258e+06
4,5,742 HYDE PARK AVE,BOSTON,35,42.275000,-71.120000,1,Inner Core,1,Metropolitan Core Communities,...,5,23,Transit-Related,32,0.000000,0.0000,0.0,0.0,-7.917042e+06,5.202263e+06


In [5]:
df['etod_qtas']

0      4
1      4
2      2
3      2
4      3
      ..
321    2
322    4
323    5
324    2
325    4
Name: etod_qtas, Length: 326, dtype: int64

In [13]:
transit_df = df[['long_merc', 'lat_merc', 'name', 'etod_qtas', 'etod_qtci','etod_qabc', 'etod_sub1t']]
p = figure(x_range=(-7992739.4389570, -7848024.1009258), y_range=(5146011.6792828, 5281579.1845907),
           x_axis_type="mercator", y_axis_type="mercator", title="MBTA communities", tools="lasso_select, box_zoom, reset")
p.xaxis.axis_label = 'Longitude'
p.yaxis.axis_label = 'Latitude'
p.add_tile(get_provider(Vendors.OSM))

s1 = ColumnDataSource(data=dict(long_merc=df['long_merc'].to_list(), 
                                lat_merc=df['lat_merc'].to_list(),
                                name=df['name'].to_list(),
                                etod_qtas=df['etod_qtas'].to_list(),
                                etod_qtci=df['etod_qtci'].to_list(),
                                etod_qabc=df['etod_qabc'].to_list()))
p.scatter(x='long_merc', y='lat_merc', size=7,  source=s1, alpha=0.6)

s2 = ColumnDataSource(data=dict(name=df['name'].to_list(), 
                                etod_qtas=df['etod_qtas'].to_list(), 
                                etod_qtci=df['etod_qtci'].to_list(), 
                                etod_qabc=df['etod_qabc'].to_list()))

factors = ['etod_qtas', 'etod_qtci', 'etod_qabc']
p2 = figure(title='Transit', x_range=transit_df['name'], tools="", )  # x_range is initially empty
vbar = p2.vbar_stack(factors, x='name', width=0.9, source=s2, color=['red','blue','green'],legend_label=[
    'land area accessible via transit, quintile ranking',
    'frequency and utilization of transit, quintile ranking',
    'percent of commuters using a mode other than driving alone, quintile ranking'])


p.js_on_event(events.Reset, CustomJS(args=dict(s1=s1, s2=s2), 
    code="""
        const d1 = s1.data;
        s2.data = {d1.name, d1.etod_qtas, d1.etod_qtci, d1.etod_qabc};
        s2.change.emit();
    """),
)
s2.js_link('name', p2, 'x_range')

s1.selected.js_on_change('indices', CustomJS(args=dict(s1=s1, s2=s2), 
    code="""
        const inds = cb_obj.indices;
        const d1 = s1.data;
        const name = Array.from(inds, (i) => d1.name[i]);
        const etod_qtas = Array.from(inds, (i) => d1.etod_qtas[i]);
        const etod_qtci = Array.from(inds, (i) => d1.etod_qtci[i]);
        const etod_qabc = Array.from(inds, (i) => d1.etod_qabc[i]); 
        s2.data = {name, etod_qtas, etod_qtci, etod_qabc};
        s2.change.emit();
        
    """),
)

layout = row(p,p2)
show(layout)
# save(layout, 'test.html')